# Health Data - Pre-processing
The Chicago Health Atlas is an open-access database for public health data. Their website includes map makers, which are convenient for quick visualizations. This pre-processing notebook demonstrates how to download and clean the raw data from the Chicago Health Atlas so that it can easily be plotted in Python. This will make it easier for the CROCUS project to do data-analysis on multiple data source (including our meteorological data).

Raw data (downloaded directly from the Chicago Health Atlas) is stored in `/heat-health-risk-cookbook/datasets/chi_health_atlas_raw`. Data that is cleaned with this notebook will then be stored in `/heat-health-risk-cookbook/datasets/chi_health_atlas_clean`.

All CHA indicators (such as adult diabetes) include a count, and a population rate. The "template" functions below merge both datasets together so that there is only one datafile per indicator.

Source: https://chicagohealthatlas.org/how-to
And here is the API: https://chicagohealthatlas.org/api/v1/

*Suggestion:* If anyone would like to automate this process through the API, that would be a great feature!

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
path = "./" # Your path to crocus_heat/datasets/
raw_data_prefix = "../../datasets/chi_health_atlas_raw/Chicago Health Atlas Data Download - "
clean_data_prefix = "../../datasets/chi_health_atlas_clean/"

### Template for community areas
Using adult diabetes in community areas as an example. For this data, you will need to identify and hand-fill the file names, attribute names, and your desired new attribute name.

Note that (as of this writing), the attributes for CHA data are always in the same order. Therefore, we can hard code which attribute names we want to replace with new names. The order is:
- "Layer"
- "GEOID"
- "Name"
- attribute name for counts (e.g. "HCSDIA_2021-2022")
- attribute name for margin of error of counts (e.g. "HCSDIA_2021-2022_moe")
- attribute name for rate (e.g. "HCSDIAP_2021-2022")
- attribute name for margin of error of rate (e.g. "HCSDIAP_2021-2022_moe")

In [47]:
# Fill these by hand for your file
file1_name = "Adult diabetes in Community Areas.csv"
file2_name = "Adult diabetes rate in Community Areas.csv"
data_name = "adult_diabetes" # This is your new file name, and the prefix for the attribute names

def clean_commareas_data(file1_name, file2_name, data_name):
    # Read in raw data and merge
    df_counts = pd.read_csv(path + raw_data_prefix + file1_name)
    df_rate = pd.read_csv(path + raw_data_prefix + file2_name)
    df = pd.merge(left=df_counts, right=df_rate, how="outer", left_on=["Layer", "GEOID", "Name"], right_on=["Layer", "GEOID", "Name"])

    # Rename columns to be more descriptive, and to match boundary shapefiles
    # For community areas, "GEOID" is the area number
    attributes = df.columns
    print(attributes)
    df.rename(columns={"Name": "community", "GEOID": "area_num",\
                        attributes[3]: data_name + "_count", attributes[4]: data_name + "_count_moe",\
                        attributes[5]: data_name + "_rate", attributes[6]: data_name + "_rate_moe"},\
              inplace=True)

    df.drop("Layer", axis=1, inplace=True)

    # Standardize community area names
    df["community"] = df["community"].str.upper()
    df.loc[df["community"] == "O'HARE", "community"] = "OHARE"

    # Drop the first row (which is informational)
    print("Dropping the following informational row")
    print(df.loc[df.index == 0])
    df.drop([0], axis=0, inplace=True)

    # Save
    df.to_csv(path + clean_data_prefix + data_name + "_commareas.csv", index=False)
    return df

df = clean_commareas_data(file1_name, file2_name, data_name)
df.head()

Index(['Layer', 'Name', 'GEOID', 'HCSDIA_2021-2022', 'HCSDIA_2021-2022_moe',
       'HCSDIAP_2021-2022', 'HCSDIAP_2021-2022_moe'],
      dtype='object')
Dropping the following informational row
  community  area_num                         adult_diabetes_count  \
0       NaN       NaN  Adult diabetes (count of adults), 2021-2022   

  adult_diabetes_count_moe                           adult_diabetes_rate  \
0    (90% margin of error)  Adult diabetes rate (% of adults), 2021-2022   

  adult_diabetes_rate_moe  
0   (90% margin of error)  


,community,area_num,adult_diabetes_count,adult_diabetes_count_moe,adult_diabetes_rate,adult_diabetes_rate_moe
1,ROGERS PARK,1.0,2600.0,1698.131024275,6.1299524989,3.953103167034
2,NORWOOD PARK,10.0,2400.0,1424.1970668534,8.5093049364,4.9318937486385
3,JEFFERSON PARK,11.0,1900.0,1599.46145264605,8.8099700233,6.8320810010145
4,FOREST GLEN,12.0,1500.0,1103.6614933792,11.359085406,7.5745882992955
5,ALBANY PARK,14.0,6300.0,3207.188527284,14.339056125,6.8486054829955005


### Template for Zip Codes
Using Asthma emergency department visits as an example.

In [48]:
# Fill these by hand for your file
file1_name = "Diabetes-related hospitalizations in ZIP Codes.csv"
file2_name = "Diabetes-related hospitalization rate in ZIP Codes.csv"
data_name = "diabetes_hosp" # This is your new file name, and the prefix for the attribute names

def clean_zip_data(file1_name, file2_name, data_name):
    # Read in raw data and merge (so that counts and rates are together)
    df_counts = pd.read_csv(path + raw_data_prefix + file1_name)
    df_rate = pd.read_csv(path + raw_data_prefix + file2_name)
    df = pd.merge(left=df_counts, right=df_rate, how="outer", left_on=["Layer", "GEOID", "Name"], right_on=["Layer", "GEOID", "Name"])

    # Rename columns to be more descriptive, and to match boundary shapefiles
    # For zip codes, "GEOID" is not needed because it is the same as the zip code
    attributes = df.columns
    df.rename(columns={"Name": "zip_code", attributes[3]: data_name + "_count",\
                       attributes[4]: data_name + "_count_moe", attributes[5]: data_name + "_rate",\
                       attributes[6]: data_name + "_rate_moe"},
              inplace=True)
    df.drop(["Layer", "GEOID"], axis=1, inplace=True)

    # Drop the first row (which is informational)
    print("Dropping the following informational row")
    print(df.loc[df.index == 0])
    df.drop([0], axis=0, inplace=True)

    # Adjust dtypes
    df = df.apply(pd.to_numeric, args=("raise", "integer"))

    # Save
    df.to_csv(path + clean_data_prefix + data_name + "_zip_codes.csv", index=False)
    return df
    
df = clean_zip_data(file1_name, file2_name, data_name)
df.head()

Dropping the following informational row
   zip_code                                diabetes_hosp_count  \
0       NaN  Diabetes-related hospitalizations (count of ho...   

  diabetes_hosp_count_moe                                 diabetes_hosp_rate  \
0   (90% margin of error)  Diabetes-related hospitalization rate (per 10,...   

  diabetes_hosp_rate_moe  
0  (90% margin of error)  


,zip_code,diabetes_hosp_count,diabetes_hosp_count_moe,diabetes_hosp_rate,diabetes_hosp_rate_moe
1,60605,12,NaN,5.6,2.769643
2,60607,28,NaN,22.8,8.015179
3,60608,166,NaN,25.7,3.441071
4,60609,188,NaN,38.1,4.741964
5,60610,34,NaN,9.7,3.021429


### Adult Asthma in Community Areas
Source info available at the following links (click "Sources")
- https://chicagohealthatlas.org/indicators/HCSATH?topic=adult-asthma
- https://chicagohealthatlas.org/indicators/HCSATHP?topic=adult-asthma-rate

In [49]:
# Adult Asthma
file1_name = "Adult asthma in Community Areas.csv"
file2_name = "Adult asthma rate in Community Areas.csv"
data_name = "adult_asthma"

adult_asthma = clean_commareas_data(file1_name, file2_name, data_name)
adult_asthma.head()

Index(['Layer', 'Name', 'GEOID', 'HCSATH_2021-2022', 'HCSATH_2021-2022_moe',
       'HCSATHP_2021-2022', 'HCSATHP_2021-2022_moe'],
      dtype='object')
Dropping the following informational row
  community  area_num                         adult_asthma_count  \
0       NaN       NaN  Adult asthma (count of adults), 2021-2022   

  adult_asthma_count_moe                           adult_asthma_rate  \
0  (90% margin of error)  Adult asthma rate (% of adults), 2021-2022   

   adult_asthma_rate_moe  
0  (90% margin of error)  


,community,area_num,adult_asthma_count,adult_asthma_count_moe,adult_asthma_rate,adult_asthma_rate_moe
1,ROGERS PARK,1.0,4900.0,2904.498116408,11.687814795,6.439035275897
2,NORWOOD PARK,10.0,3000.0,1625.9889338147,10.697054662,5.54126845783
3,JEFFERSON PARK,11.0,600.0,325.2073188273,2.4396925606,1.429608704748
4,FOREST GLEN,12.0,1100.0,740.95782808635,8.6175984924,5.437838144522001
5,ALBANY PARK,14.0,3500.0,2073.3267721425,8.0608019141,4.6966871541695


### Adult Diabetes in Community Areas
Source info available at the following links (click "Sources")
- https://chicagohealthatlas.org/indicators/HCSDIA?topic=adult-diabetes
- https://chicagohealthatlas.org/indicators/HCSDIAP?topic=adult-diabetes-rate

In [50]:
# Adult Diabetes
file1_name = "Adult diabetes in Community Areas.csv"
file2_name = "Adult diabetes rate in Community Areas.csv"
data_name = "adult_diabetes"

adult_diabetes = clean_commareas_data(file1_name, file2_name, data_name)
adult_diabetes.head()

Index(['Layer', 'Name', 'GEOID', 'HCSDIA_2021-2022', 'HCSDIA_2021-2022_moe',
       'HCSDIAP_2021-2022', 'HCSDIAP_2021-2022_moe'],
      dtype='object')
Dropping the following informational row
  community  area_num                         adult_diabetes_count  \
0       NaN       NaN  Adult diabetes (count of adults), 2021-2022   

  adult_diabetes_count_moe                           adult_diabetes_rate  \
0    (90% margin of error)  Adult diabetes rate (% of adults), 2021-2022   

  adult_diabetes_rate_moe  
0   (90% margin of error)  


,community,area_num,adult_diabetes_count,adult_diabetes_count_moe,adult_diabetes_rate,adult_diabetes_rate_moe
1,ROGERS PARK,1.0,2600.0,1698.131024275,6.1299524989,3.953103167034
2,NORWOOD PARK,10.0,2400.0,1424.1970668534,8.5093049364,4.9318937486385
3,JEFFERSON PARK,11.0,1900.0,1599.46145264605,8.8099700233,6.8320810010145
4,FOREST GLEN,12.0,1500.0,1103.6614933792,11.359085406,7.5745882992955
5,ALBANY PARK,14.0,6300.0,3207.188527284,14.339056125,6.8486054829955005


### Adult Loneliness in Community Areas
Source info available at the following links (click "Sources")
- https://chicagohealthatlas.org/indicators/CHAJVUB?topic=adult-loneliness
- https://chicagohealthatlas.org/indicators/CHAVQOS?topic=adult-loneliness-rate

In [51]:
# Adult loneliness
file1_name = "Adult loneliness in Community Areas.csv"
file2_name = "Adult loneliness rate in Community Areas.csv"
data_name = "adult_loneliness"

adult_loneliness = clean_commareas_data(file1_name, file2_name, data_name)
adult_loneliness.head()

Index(['Layer', 'Name', 'GEOID', 'CHAJVUB_2021-2022', 'CHAJVUB_2021-2022_moe',
       'CHAVQOS_2021-2022', 'CHAVQOS_2021-2022_moe'],
      dtype='object')
Dropping the following informational row
  community  area_num                         adult_loneliness_count  \
0       NaN       NaN  Adult loneliness (count of adults), 2021-2022   

  adult_loneliness_count_moe  \
0      (90% margin of error)   

                               adult_loneliness_rate adult_loneliness_rate_moe  
0  Adult loneliness rate (percent of adults), 202...     (90% margin of error)  


,community,area_num,adult_loneliness_count,adult_loneliness_count_moe,adult_loneliness_rate,adult_loneliness_rate_moe
1,ROGERS PARK,1.0,15800.0,4593.66646445,37.00320243,8.461179603415001
2,NORWOOD PARK,10.0,5400.0,2413.303134165,18.9093925,7.5314949878699995
3,JEFFERSON PARK,11.0,6800.0,2829.7602895450004,27.75649933,10.08983136476
4,FOREST GLEN,12.0,2600.0,881.87490965,19.13364246,6.3270902514400005
5,NORTH PARK,13.0,3300.0,1734.391308545,23.46990757,11.60819528715


### Asthma Emergency Department Visits in Zip Codes
Source info available at the following links (click "Sources")
- https://chicagohealthatlas.org/indicators/HDED?topic=asthma-ed-visits
- https://chicagohealthatlas.org/indicators/HDEDR?topic=asthma-ed-visit-rate

In [52]:
# Asthma Emergency Department Visits
file1_name = "Asthma ED visits in ZIP Codes.csv"
file2_name = "Asthma ED visit rate in ZIP Codes.csv"
data_name = "asthma_ed" # This is your new file name, and the prefix for the attribute names

asthma_ed = clean_zip_data(file1_name, file2_name, data_name)
asthma_ed.head()

Dropping the following informational row
   zip_code                                    asthma_ed_count  \
0       NaN  Asthma ED visits (count of hospitalizations), ...   

     asthma_ed_count_moe                                     asthma_ed_rate  \
0  (90% margin of error)  Asthma ED visit rate (per 10,000 population), ...   

      asthma_ed_rate_moe  
0  (90% margin of error)  


,zip_code,asthma_ed_count,asthma_ed_count_moe,asthma_ed_rate,asthma_ed_rate_moe
1,60601,19,NaN,51.8,26.185714
2,60604,18,NaN,388.2,234.286607
3,60605,43,NaN,29.1,9.483929
4,60606,19,NaN,352.3,224.928571
5,60607,113,NaN,76.0,14.016071


### Behavioral Health Hospitalizations in Zip Codes
Source info available at the following links (click "Sources")
- https://chicagohealthatlas.org/indicators/HDBH?topic=behavioral-health-hosp
- https://chicagohealthatlas.org/indicators/HDBHR?topic=behavioral-health-hospitalization-rate

In [53]:
# Behavioral Health Hospitalizations
file1_name = "Behavioral health hospitalizations in ZIP Codes.csv"
file2_name = "Behavioral health hospitalization rate in ZIP Codes.csv"
data_name = "behavioral_health_hosp" # This is your new file name, and the prefix for the attribute names

behavioral_hosp = clean_zip_data(file1_name, file2_name, data_name)
behavioral_hosp.head()

Dropping the following informational row
   zip_code                       behavioral_health_hosp_count  \
0       NaN  Behavioral health hospitalizations (count of h...   

  behavioral_health_hosp_count_moe  \
0            (90% margin of error)   

                         behavioral_health_hosp_rate  \
0  Behavioral health hospitalization rate (per 10...   

  behavioral_health_hosp_rate_moe  
0           (90% margin of error)  


,zip_code,behavioral_health_hosp_count,behavioral_health_hosp_count_moe,behavioral_health_hosp_rate,behavioral_health_hosp_rate_moe
1,60601,91,NaN,85.4,17.415179
2,60602,25,NaN,400.5,201.134821
3,60603,18,NaN,990.3,812.428571
4,60604,28,NaN,674.2,293.078571
5,60605,285,NaN,119.7,14.100000


### Behavioral Health Treatment in Community Areas
Source info available at the following links (click "Sources")
- https://chicagohealthatlas.org/indicators/HCSBH?topic=behavioral-health-treatment
- https://chicagohealthatlas.org/indicators/HCSBHP?topic=behavioral-health-treatment-rate

In [54]:
# Behavioral health treatment
file1_name = "Behavioral health treatment in Community Areas.csv"
file2_name = "Behavioral health treatment rate in Community Areas.csv"
data_name = "behavioral_health_treatment" # This is your new file name, and the prefix for the attribute names

behavioral_treatment = clean_commareas_data(file1_name, file2_name, data_name)
behavioral_treatment.head()

Index(['Layer', 'Name', 'GEOID', 'HCSBH_2021-2022', 'HCSBH_2021-2022_moe',
       'HCSBHP_2021-2022', 'HCSBHP_2021-2022_moe'],
      dtype='object')
Dropping the following informational row
  community  area_num                  behavioral_health_treatment_count  \
0       NaN       NaN  Behavioral health treatment (count of adults),...   

  behavioral_health_treatment_count_moe  \
0                 (90% margin of error)   

                    behavioral_health_treatment_rate  \
0  Behavioral health treatment rate (% of adults)...   

  behavioral_health_treatment_rate_moe  
0                (90% margin of error)  


,community,area_num,behavioral_health_treatment_count,behavioral_health_treatment_count_moe,behavioral_health_treatment_rate,behavioral_health_treatment_rate_moe
1,ROGERS PARK,1.0,3600.0,2380.2617914879997,49.81188652,23.258801679715
2,JEFFERSON PARK,11.0,500.0,400.78901338490004,35.996216943,26.833910949440003
3,FOREST GLEN,12.0,200.0,223.10472987845003,42.74802593,28.946620840130002
4,PORTAGE PARK,15.0,1600.0,1586.1846473648,45.392527743,30.97833716208
5,IRVING PARK,16.0,3000.0,1927.2340666740001,66.575342639,21.68128851042


### Diabetes-related hospitalizations by Zip Code
Source info available at the following links (click "Sources")
- https://chicagohealthatlas.org/indicators/HDDIA?topic=diabetes-related-hosp
- https://chicagohealthatlas.org/indicators/HDDIAR?topic=diabetes-related-hospitalization-rate

In [55]:
# Diabetes related hospitalizations
file1_name = "Diabetes-related hospitalizations in ZIP Codes.csv"
file2_name = "Diabetes-related hospitalization rate in ZIP Codes.csv"
data_name = "diabetes_hosp" # This is your new file name, and the prefix for the attribute names

diabetes_hosp = clean_zip_data(file1_name, file2_name, data_name)
diabetes_hosp.head()

Dropping the following informational row
   zip_code                                diabetes_hosp_count  \
0       NaN  Diabetes-related hospitalizations (count of ho...   

  diabetes_hosp_count_moe                                 diabetes_hosp_rate  \
0   (90% margin of error)  Diabetes-related hospitalization rate (per 10,...   

  diabetes_hosp_rate_moe  
0  (90% margin of error)  


,zip_code,diabetes_hosp_count,diabetes_hosp_count_moe,diabetes_hosp_rate,diabetes_hosp_rate_moe
1,60605,12,NaN,5.6,2.769643
2,60607,28,NaN,22.8,8.015179
3,60608,166,NaN,25.7,3.441071
4,60609,188,NaN,38.1,4.741964
5,60610,34,NaN,9.7,3.021429


### Hypertension in Community Areas
Source info available at the following links (click "Sources")
- https://chicagohealthatlas.org/indicators/HCSHYT?topic=hypertension
- https://chicagohealthatlas.org/indicators/HCSHYTP?topic=hypertension-rate

In [57]:
# Hypertension
file1_name = "Hypertension in Community Areas.csv"
file2_name = "Hypertension rate in Community Areas.csv"
data_name = "hypertension" # This is your new file name, and the prefix for the attribute names

hypertension = clean_commareas_data(file1_name, file2_name, data_name)
hypertension.head()

Index(['Layer', 'Name', 'GEOID', 'HCSHYT_2021-2022', 'HCSHYT_2021-2022_moe',
       'HCSHYTP_2021-2022', 'HCSHYTP_2021-2022_moe'],
      dtype='object')
Dropping the following informational row
  community  area_num                         hypertension_count  \
0       NaN       NaN  Hypertension (count of adults), 2021-2022   

  hypertension_count_moe                           hypertension_rate  \
0  (90% margin of error)  Hypertension rate (% of adults), 2021-2022   

   hypertension_rate_moe  
0  (90% margin of error)  


,community,area_num,hypertension_count,hypertension_count_moe,hypertension_rate,hypertension_rate_moe
1,ROGERS PARK,1.0,13700.0,4208.630089538,32.177739098,8.151799908609
2,NORWOOD PARK,10.0,9400.0,2770.0579980815,32.83628215,8.4060731586825
3,JEFFERSON PARK,11.0,6900.0,2573.565055194,27.920677805,9.597104749501499
4,FOREST GLEN,12.0,3700.0,1307.64147635345,27.399344012,8.36863805736
5,NORTH PARK,13.0,3300.0,1206.2794207065,23.817396641,9.779921678096501


### Mood and Depressive Disorder Related Hospitalizations by Zip Code
Source info available at the following links (click "Sources")
- https://chicagohealthatlas.org/indicators/HDMD?topic=mood-and-depressive-disorder-hosp
- https://chicagohealthatlas.org/indicators/HDMDR?topic=mood-and-depressive-disorder-hospitalization-rate

In [58]:
# Mood and depressive disorder hosp
file1_name = "Mood and depressive disorder hospitalizations in ZIP Codes.csv"
file2_name = "Mood and depressive disorder hospitalization rate in ZIP Codes.csv"
data_name = "mood_hosp" # This is your new file name, and the prefix for the attribute names

mood_hosp = clean_zip_data(file1_name, file2_name, data_name)
mood_hosp.head()

Dropping the following informational row
   zip_code                                    mood_hosp_count  \
0       NaN  Mood and depressive disorder hospitalizations ...   

     mood_hosp_count_moe                                     mood_hosp_rate  \
0  (90% margin of error)  Mood and depressive disorder hospitalization r...   

      mood_hosp_rate_moe  
0  (90% margin of error)  


,zip_code,mood_hosp_count,mood_hosp_count_moe,mood_hosp_rate,mood_hosp_rate_moe
1,60601,36,NaN,35.1,12.715179
2,60604,12,NaN,167.1,84.138393
3,60605,104,NaN,44.7,9.106250
4,60606,20,NaN,105.5,55.015179
5,60607,227,NaN,99.5,12.463393


### Schizophrenic Disorder Related Hospitalizations by Zip Code
Source info available at the following links (click "Sources")
- https://chicagohealthatlas.org/indicators/HDSZ?topic=schizophrenic-disorder-hosp
- https://chicagohealthatlas.org/indicators/HDSZR?topic=schizophrenic-disorder-hospitalization-rate

In [59]:
# Schizophrenic disorder hospitalizations
file1_name = "Schizophrenic disorder hospitalizations in ZIP Codes.csv"
file2_name = "Schizophrenic disorder hospitalization rate in ZIP Codes.csv"
data_name = "schizophrenia_hosp" # This is your new file name, and the prefix for the attribute names

schizophrenia_hosp = clean_zip_data(file1_name, file2_name, data_name)
schizophrenia_hosp.head()

Dropping the following informational row
   zip_code                           schizophrenia_hosp_count  \
0       NaN  Schizophrenic disorder hospitalizations (count...   

  schizophrenia_hosp_count_moe  \
0        (90% margin of error)   

                             schizophrenia_hosp_rate  \
0  Schizophrenic disorder hospitalization rate (p...   

  schizophrenia_hosp_rate_moe  
0       (90% margin of error)  


,zip_code,schizophrenia_hosp_count,schizophrenia_hosp_count_moe,schizophrenia_hosp_rate,schizophrenia_hosp_rate_moe
1,60601,17,NaN,16.7,6.966071
2,60605,75,NaN,30.8,6.042857
3,60606,23,NaN,80.8,30.550000
4,60607,222,NaN,106.6,12.631250
5,60608,237,NaN,28.5,3.147321


### Serious Psychological Distress by Community Area
Source info available at the following links (click "Sources")
- https://chicagohealthatlas.org/indicators/HCSSPD?topic=serious-psychological-distress
- https://chicagohealthatlas.org/indicators/HCSSPDP?topic=serious-psychological-distress-rate

In [60]:
# Serious psychological distress
file1_name = "Serious psychological distress in Community Areas.csv"
file2_name = "Serious psychological distress rate in Community Areas.csv"
data_name = "psych_distress" # This is your new file name, and the prefix for the attribute names

psych_distress = clean_commareas_data(file1_name, file2_name, data_name)
psych_distress.head()

Index(['Layer', 'Name', 'GEOID', 'HCSSPD_2021-2022', 'HCSSPD_2021-2022_moe',
       'HCSSPDP_2021-2022', 'HCSSPDP_2021-2022_moe'],
      dtype='object')
Dropping the following informational row
  community  area_num                               psych_distress_count  \
0       NaN       NaN  Serious psychological distress (count of adult...   

  psych_distress_count_moe                                psych_distress_rate  \
0    (90% margin of error)  Serious psychological distress rate (% of adul...   

  psych_distress_rate_moe  
0   (90% margin of error)  


,community,area_num,psych_distress_count,psych_distress_count_moe,psych_distress_rate,psych_distress_rate_moe
1,ROGERS PARK,1.0,7300.0,3398.8723424460004,17.175378928,7.1236505723145
2,NORWOOD PARK,10.0,1700.0,1421.0400948285499,5.869389009,4.739661113936
3,JEFFERSON PARK,11.0,1300.0,753.6732034709499,5.210042634,3.1446979460155
4,FOREST GLEN,12.0,600.0,325.14061340285,4.1883017813,2.4478224710135
5,ALBANY PARK,14.0,6000.0,3999.0807041710004,13.10642455,8.0084084721165
